# 2.1:  Computer Tomography: Forward Transform
Gruppe: Gabriel Wolf und Jakob Scheiber

### Unterpunkt a: Winkelbereich und Definitionsbereiche

Zuerst ist der Bereich, in dem $\varphi$ sein kann, gesucht. 

Mit einem Blick auf die Grafik 1 in der Angabe wird klar das sich der Winkel

zwischen 0° und exklusive 180° befinden kann.

Somit folgt $\varphi \in [0;180)$°